## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# enable extensions of jupyter
!jupyter nbextension enable --py gmaps
!enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
'enable' is not recognized as an internal or external command,
operable program or batch file.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [4]:
# 1. Import the WeatherPy_database.csv file and keep "NA" in Country column for Namibia
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv", keep_default_na=False, na_values=[''])
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Narsaq,GL,60.9167,-46.0500,48.63,100,100,2.91,heavy intensity rain
1,1,Saint-Francois,FR,46.4154,3.9054,47.64,100,20,0.00,fog
2,2,Port Alfred,ZA,-33.5906,26.8910,59.18,90,100,1.99,overcast clouds
3,3,Mar Del Plata,AR,-38.0023,-57.5575,55.09,97,100,6.91,fog
4,4,Punta Arenas,CL,-53.1500,-70.9167,44.71,76,75,5.75,broken clouds
5,5,Ushuaia,AR,-54.8000,-68.3000,46.06,57,75,8.05,broken clouds
6,6,Sinnamary,GF,5.3833,-52.9500,79.92,83,0,12.71,clear sky
7,7,Klaksvik,FO,62.2266,-6.5890,41.92,95,100,51.47,light rain
8,8,Korla,CN,41.7597,86.1469,69.69,35,57,3.18,broken clouds
9,9,Kulhudhuffushi,MV,6.6221,73.0700,82.29,75,79,12.39,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Sinnamary,GF,5.3833,-52.9500,79.92,83,0,12.71,clear sky
9,9,Kulhudhuffushi,MV,6.6221,73.0700,82.29,75,79,12.39,broken clouds
11,11,Hithadhoo,MV,-0.6000,73.0833,81.00,75,99,13.71,overcast clouds
13,13,Rojhan,PK,28.6868,69.9534,81.41,58,0,8.08,clear sky
16,16,Mattru,SL,7.6244,-11.8332,72.45,97,100,2.30,moderate rain
18,18,Touros,BR,-5.1989,-35.4608,76.77,79,0,13.58,clear sky
19,19,Maceio,BR,-9.6658,-35.7353,72.84,94,20,2.30,few clouds
30,30,Cayenne,GF,4.9333,-52.3333,78.84,89,0,5.75,clear sky
31,31,Ponta Do Sol,PT,32.6667,-17.1000,70.72,73,11,8.68,few clouds
32,32,Kismayo,SO,-0.3582,42.5454,76.73,87,6,13.00,clear sky


In [7]:
# 4a. Determine if there are any empty rows
preferred_cities_df.count()

City_ID                262
City                   262
Country                262
Lat                    262
Lng                    262
Max Temp               262
Humidity               262
Cloudiness             262
Wind Speed             262
Current Description    262
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                262
City                   262
Country                262
Lat                    262
Lng                    262
Max Temp               262
Humidity               262
Cloudiness             262
Wind Speed             262
Current Description    262
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Sinnamary,GF,79.92,clear sky,5.3833,-52.9500,
9,Kulhudhuffushi,MV,82.29,broken clouds,6.6221,73.0700,
11,Hithadhoo,MV,81.00,overcast clouds,-0.6000,73.0833,
13,Rojhan,PK,81.41,clear sky,28.6868,69.9534,
16,Mattru,SL,72.45,moderate rain,7.6244,-11.8332,
18,Touros,BR,76.77,clear sky,-5.1989,-35.4608,
19,Maceio,BR,72.84,few clouds,-9.6658,-35.7353,
30,Cayenne,GF,78.84,clear sky,4.9333,-52.3333,
31,Ponta Do Sol,PT,70.72,few clouds,32.6667,-17.1000,
32,Kismayo,SO,76.73,clear sky,-0.3582,42.5454,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel found")
    except (IndexError):
        print("Hotel not found... skipping.")
        
# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found... skipping.
Hotel found
Hotel found
Hotel found
Hotel not found... skipping.
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found... skipping.
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel not found... skipping.
Hotel fo

In [11]:
# Determine if there are any empty rows
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.count()

City                   241
Country                241
Max Temp               241
Current Description    241
Lat                    241
Lng                    241
Hotel Name             241
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.count()

City                   241
Country                241
Max Temp               241
Current Description    241
Lat                    241
Lng                    241
Hotel Name             241
dtype: int64

In [13]:
# Checking clean_hotel dataframe
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Sinnamary,GF,79.92,clear sky,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
9,Kulhudhuffushi,MV,82.29,broken clouds,6.6221,73.0700,Haajy Guest House 1
11,Hithadhoo,MV,81.00,overcast clouds,-0.6000,73.0833,Scoop Guest House
13,Rojhan,PK,81.41,clear sky,28.6868,69.9534,Nawab Imam Bux Khan Bangla
16,Mattru,SL,72.45,moderate rain,7.6244,-11.8332,"Mondkor Town, Bagbo Chiefdom, Bo District"
...,...,...,...,...,...,...,...
676,Sumbe,AO,70.92,overcast clouds,-11.2061,13.8437,Hotel Ritz Sumbe
677,Awbari,LY,80.65,clear sky,26.5921,12.7805,استراحة الشارب العائلي
678,Mtwara,TZ,72.57,scattered clouds,-10.2667,40.1833,TIL Hotels Mtwara
680,Oistins,BB,80.69,few clouds,13.0667,-59.5333,Butterfly Beach Hotel


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(50.0, -50.0), zoom_level=2.0)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))